In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [2]:
spark

In [3]:
badges = spark.read.parquet('/data/stackoverflow/Badges')

In [4]:
badges.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- UserId: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Class: integer (nullable = true)
 |-- TagBased: boolean (nullable = true)



In [5]:
badges.count()

32450580

## Odczyt danych

In [6]:
badges.first()

Row(Id=82946, UserId=3718, Name='Teacher', Date=datetime.datetime(2008, 9, 15, 8, 55, 3, 923000), Class=3, TagBased=False)

In [7]:
badges.first().Name == 'Teacher'

True

In [8]:
badges.show(3)

+-----+------+-------+--------------------+-----+--------+
|   Id|UserId|   Name|                Date|Class|TagBased|
+-----+------+-------+--------------------+-----+--------+
|82946|  3718|Teacher|2008-09-15 08:55:...|    3|   false|
|82947|   994|Teacher|2008-09-15 08:55:...|    3|   false|
|82949|  3893|Teacher|2008-09-15 08:55:...|    3|   false|
+-----+------+-------+--------------------+-----+--------+
only showing top 3 rows



In [9]:
badges.show(3, truncate=False)

+-----+------+-------+-----------------------+-----+--------+
|Id   |UserId|Name   |Date                   |Class|TagBased|
+-----+------+-------+-----------------------+-----+--------+
|82946|3718  |Teacher|2008-09-15 08:55:03.923|3    |false   |
|82947|994   |Teacher|2008-09-15 08:55:03.957|3    |false   |
|82949|3893  |Teacher|2008-09-15 08:55:03.957|3    |false   |
+-----+------+-------+-----------------------+-----+--------+
only showing top 3 rows



In [10]:
first_5_records = badges.take(5)
first_5_records[-1]

Row(Id=82951, UserId=5196, Name='Teacher', Date=datetime.datetime(2008, 9, 15, 8, 55, 3, 957000), Class=3, TagBased=False)

In [11]:
badges.limit(300).toPandas()

,Id,UserId,Name,Date,Class,TagBased
0,82946,3718,Teacher,2008-09-15 08:55:03.923,3,False
1,82947,994,Teacher,2008-09-15 08:55:03.957,3,False
2,82949,3893,Teacher,2008-09-15 08:55:03.957,3,False
3,82950,4591,Teacher,2008-09-15 08:55:03.957,3,False
4,82951,5196,Teacher,2008-09-15 08:55:03.957,3,False
...,...,...,...,...,...,...
295,83421,3289,Teacher,2008-09-15 08:55:03.957,3,False
296,83422,2688,Teacher,2008-09-15 08:55:03.957,3,False
297,83423,3622,Teacher,2008-09-15 08:55:03.957,3,False
298,83424,698,Teacher,2008-09-15 08:55:03.957,3,False


In [12]:
badges_with_limit = badges.limit(2)
badges_with_limit.collect()

[Row(Id=82946, UserId=3718, Name='Teacher', Date=datetime.datetime(2008, 9, 15, 8, 55, 3, 923000), Class=3, TagBased=False),
 Row(Id=82947, UserId=994, Name='Teacher', Date=datetime.datetime(2008, 9, 15, 8, 55, 3, 957000), Class=3, TagBased=False)]

In [13]:
badges.take(2)

[Row(Id=82946, UserId=3718, Name='Teacher', Date=datetime.datetime(2008, 9, 15, 8, 55, 3, 923000), Class=3, TagBased=False),
 Row(Id=82947, UserId=994, Name='Teacher', Date=datetime.datetime(2008, 9, 15, 8, 55, 3, 957000), Class=3, TagBased=False)]

## Selekcja danych (`WHERE`)

In [14]:
condition = 'Class = 1' # 1 means gold badge
badges.where(condition).count()

892514

In [15]:
condition = badges.Class == 1
print(condition)
badges.where(condition).count()

Column<b'(Class = 1)'>


892514

In [16]:
condition = col('Class') == 1
print(condition)
badges.where(condition).count()

Column<b'(Class = 1)'>


892514

In [17]:
badges.where(condition).first()

Row(Id=100568, UserId=30, Name='Great Question', Date=datetime.datetime(2008, 9, 15, 8, 55, 13, 720000), Class=1, TagBased=False)

In [18]:
gold_badge = col('Class') == 1
silver_badgs = col('Class') == 2
badges.where(gold_badge | silver_badgs).count()

7473443

In [19]:
gold_or_silber_badge = col('Class').isin(1, 2)
badges.where(gold_or_silber_badge).count()

7473443

In [20]:
not_gold_or_silber_badge = ~col('Class').isin(1, 2) # ticket no
badges.where(gold_or_silber_badge).count()

7473443

In [21]:
badges.where(col('Class') == 1).count()

892514

In [22]:
badges.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- UserId: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Class: integer (nullable = true)
 |-- TagBased: boolean (nullable = true)



In [23]:
badges.where(col('TagBased')).count()
badges.where(col('TagBased') == True).count()
badges.where('TagBased').count()

179785

In [24]:
badges.where(~col('TagBased')).count()

32270795

In [25]:
badges.where(col('TagBased').isNull()).count()

0

In [26]:
badges.where(col('TagBased').isNotNull()).count()

32450580

In [27]:
badges

DataFrame[Id: int, UserId: int, Name: string, Date: timestamp, Class: int, TagBased: boolean]

In [28]:
badges.where(col('Name') == 'Teacher').count()

1457324

In [29]:
badges.where(col('Name').like('T%')).count()

2715940

In [30]:
badges.where(col('Name').rlike('^[Tt]')).count()

2718934

In [31]:
%%time
badges.where(col('name') == 'Teacher').count()

CPU times: user 3.75 ms, sys: 3.19 ms, total: 6.94 ms
Wall time: 1.03 s


1457324

In [32]:
%%time
badges.where(lower(col('name')) == 'teacher').count()

CPU times: user 2.61 ms, sys: 3.26 ms, total: 5.87 ms
Wall time: 1.64 s


1457324

In [33]:
badges

DataFrame[Id: int, UserId: int, Name: string, Date: timestamp, Class: int, TagBased: boolean]

In [34]:
badges.where(year('Date') == '2019').count()

2906724

In [35]:
badges.where(col('Date').cast('date').between('2019-01-10', '2019-01-17')).count()

91546

`YYYY-MM-DD HH:MM:SS.sss`

In [36]:
badges.where(col('Date').cast('date').between('2019-01-10', '2019-01-17')).first()

Row(Id=30881586, UserId=10895603, Name='Editor', Date=datetime.datetime(2019, 1, 14, 10, 30, 14, 73000), Class=3, TagBased=False)

## Projekcja danych (`SELECT`)

In [37]:
badges.select('Name').show()

+-------+
|   Name|
+-------+
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
|Teacher|
+-------+
only showing top 20 rows



In [38]:
columns_to_select = ['UserId', 'Name']
badges.select(columns_to_select).show(10)

+------+-------+
|UserId|   Name|
+------+-------+
|  3718|Teacher|
|   994|Teacher|
|  3893|Teacher|
|  4591|Teacher|
|  5196|Teacher|
|  2635|Teacher|
|  1113|Teacher|
|  4182|Teacher|
|   164|Teacher|
|   652|Teacher|
+------+-------+
only showing top 10 rows



#### pokaż 10 wierszy UserId + Name, gdzie name to nie "Teacher"

1. `!=`: `col('Name') != "Teacher"`
2. `~(col('Name') == "Teacher")`
3. `(col('Name') > "Teacher") | (col('Name') < "Teacher")`

Kolejność

1. `select().where()`
2. `where().select()`

In [39]:
badges.where(col('Name')!='Teacher').select('UserId', 'Name').explain()

== Physical Plan ==
*(1) Project [UserId#1, Name#2]
+- *(1) Filter (isnotnull(Name#2) && NOT (Name#2 = Teacher))
   +- *(1) FileScan parquet [UserId#1,Name#2] Batched: true, Format: Parquet, Location: InMemoryFileIndex[hdfs://namenode:9000/data/stackoverflow/Badges], PartitionFilters: [], PushedFilters: [IsNotNull(Name), Not(EqualTo(Name,Teacher))], ReadSchema: struct<UserId:int,Name:string>


In [40]:
df_with_new_columns = badges.withColumn('is_gold', col('Class') == 1) \
    .withColumn('year', year('Date'))

In [41]:
df_with_new_columns.where(col('is_gold')).count()

892514

In [42]:
df_with_new_columns.drop('Date').first()

Row(Id=82946, UserId=3718, Name='Teacher', Class=3, TagBased=False, is_gold=False, year=2008)

In [43]:
df_with_new_columns.select('Id', 'UserId', 'year', month('Date').alias('month')).first()

Row(Id=82946, UserId=3718, year=2008, month=9)

## Agregacje

In [44]:
badges.select('Name').distinct().count()

5567

In [45]:
badges.select('Name').distinct().show()

+--------------------+
|                Name|
+--------------------+
|              bazaar|
|              iframe|
|             jbutton|
|    android-manifest|
|           standards|
|               input|
|     language-lawyer|
|           sitecore8|
|                rxjs|
|                 avx|
|hyperledger-composer|
|             mailkit|
|                xlrd|
|           arguments|
|                 fog|
|    fluent-interface|
|        google-earth|
|     instruction-set|
|       webdriverwait|
|      caliburn.micro|
+--------------------+
only showing top 20 rows



In [46]:
badges.where(~col('TagBased')).select('Name').distinct().count()

91

In [47]:
# znajdź najpopularniejsze badge ogólne

In [48]:
badges.where(~col('TagBased')).groupBy('Name').agg(count('*')).show()

+-----------------+--------+
|             Name|count(1)|
+-----------------+--------+
|      Commentator|  914556|
|       Electorate|   20574|
|        Explainer|   63978|
|       Civic Duty|   94622|
|       Tumbleweed| 1156958|
|             Guru|  130077|
|    Peer Pressure|  224148|
|      Mortarboard|   31565|
|          Revival|  388347|
|          Student| 2257156|
|    Nice Question|  581899|
|       Enthusiast|  215938|
|    Archaeologist|    2200|
|       Benefactor|   43252|
|          Sheriff|      43|
|          Fanatic|   32558|
|          Teacher| 1457324|
|Favorite Question|   50756|
|        Excavator|  136997|
|           Census|  127122|
+-----------------+--------+
only showing top 20 rows



In [49]:
badges.where(~col('TagBased')).groupBy('Name').count().show()

+-----------------+-------+
|             Name|  count|
+-----------------+-------+
|      Commentator| 914556|
|       Electorate|  20574|
|        Explainer|  63978|
|       Civic Duty|  94622|
|       Tumbleweed|1156958|
|             Guru| 130077|
|    Peer Pressure| 224148|
|      Mortarboard|  31565|
|          Revival| 388347|
|          Student|2257156|
|    Nice Question| 581899|
|       Enthusiast| 215938|
|    Archaeologist|   2200|
|       Benefactor|  43252|
|          Sheriff|     43|
|          Fanatic|  32558|
|          Teacher|1457324|
|Favorite Question|  50756|
|        Excavator| 136997|
|           Census| 127122|
+-----------------+-------+
only showing top 20 rows



In [50]:
badges.sort?

In [51]:
spark.sparkContext.setJobGroup('aaa', 'bbb') # for nicer descriptions in Spark UI

In [52]:
badges.where(~col('TagBased')).groupBy('Name').count().orderBy(desc('count')).show()

+----------------+-------+
|            Name|  count|
+----------------+-------+
|Popular Question|4526111|
|          Editor|2297538|
|         Student|2257156|
|Notable Question|2230290|
|        Informed|2179709|
|         Scholar|1847021|
|        Yearling|1677778|
|         Teacher|1457324|
|       Supporter|1352041|
|     Nice Answer|1200408|
|      Tumbleweed|1156958|
|  Autobiographer| 953335|
|     Commentator| 914556|
|       Custodian| 826981|
|          Caucus| 664845|
| Famous Question| 645355|
|   Nice Question| 581899|
|     Necromancer| 567598|
|     Good Answer| 411794|
|         Revival| 388347|
+----------------+-------+
only showing top 20 rows



In [53]:
badges.agg(max('Date')).first()

Row(max(Date)=datetime.datetime(2019, 9, 1, 8, 35, 21, 237000))

In [54]:
# ile było badgy w każdym roku?
